In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
features=pd.read_csv('training_set_features.csv')
labels=pd.read_csv('training_set_labels.csv')

In [3]:
labels1=labels.drop('seasonal_vaccine',axis=1)

In [4]:
data=features.merge(labels1,how='outer')

In [5]:
data.drop(['health_insurance','employment_industry','employment_occupation','respondent_id','hhs_geo_region'],axis=1,inplace=True)

In [6]:
X=data.drop('xyz_vaccine',axis=1)
y=data['xyz_vaccine']

In [10]:
best_labels=['xyz_concern',
 'xyz_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_xyz',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'opinion_xyz_vacc_effective',
 'opinion_xyz_risk',
 'opinion_xyz_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'census_msa',
 'household_adults',
 'household_children']

In [11]:
X_new=X[best_labels]

In [12]:
num_cols_new=[i for i in X_new if X_new[i].dtype!=object]
cat_cols_new=[i for i in X_new if X_new[i].dtype==object]

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import make_column_transformer

In [14]:
num_pipe=make_pipeline(SimpleImputer(strategy='most_frequent'))
nominal_pipe=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder())
ordinal_pipe=make_pipeline(SimpleImputer(strategy='most_frequent'),OrdinalEncoder(categories=[['18 - 34 Years','35 - 44 Years','45 - 54 Years','55 - 64 Years','65+ Years'],['< 12 Years', '12 Years', 'Some College','College Graduate'],['Below Poverty','<= $75,000, Above Poverty','> $75,000'],['Unemployed','Not in Labor Force','Employed']]))

In [15]:
col_trans=make_column_transformer((num_pipe,num_cols_new),(nominal_pipe,['race','sex','marital_status','rent_or_own','census_msa']),(ordinal_pipe,['age_group','education','income_poverty','employment_status']),remainder='passthrough')

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
pipe=make_pipeline(col_trans,RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=5,
                       n_estimators=5000, n_jobs=-1, random_state=42))

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
cross_val_score(pipe,X_new,y,cv=10,scoring='roc_auc').mean()

0.8339663631296468

In [20]:
pipe.fit(X_new,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['xyz_concern',
                                                   'xyz_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavio...
                                                                                               'College '
                                                                                               'Graduate'],
                                                                                              ['Below '
                                                                                               'Poverty',
                                                                                               '<= '
                                                                                               '$75,000, '
                                                                                               'Above '
                                                                                               'Poverty',
                                                                                               '> '
                                                                                               '$75,000'],
                                                                                              ['Unemployed',
                                                                                               'Not '
                                                                                               'in '
                                                                                               'Labor '
                                                                                               'Force',
                                                                                               'Employed']]))]),
                                                  ['age_group', 'education',
                                                   'income_poverty',
                                                   'employment_status'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=20,
                                        min_samples_leaf=5, n_estimators=5000,
                                        n_jobs=-1, random_state=42))])

In [25]:
test_data=pd.read_csv('test_set_features.csv')

In [27]:
test=test_data.drop(['health_insurance','employment_industry','employment_occupation','respondent_id','hhs_geo_region'],axis=1)

In [36]:
prob=pipe.predict_proba(test)[:,1]

In [37]:
h1n1=pd.DataFrame(prob,columns=['h1n1_vaccine'])

In [39]:
h1n1.to_csv('hini_vaccine.csv')

In [40]:
h1n1

,h1n1_vaccine
0,0.117669
1,0.035440
2,0.515479
3,0.572180
4,0.258493
...,...
26703,0.388792
26704,0.144376
26705,0.080795
26706,0.060990
